# Error checking calcs

Hannah and Joe both checked 10% of lines from data.

In [ ]:
# setwd("ORIGINAL PATH REMOVED")
library(compare)

In [ ]:
# Percentages of change
precheck_data <- read.csv(file='backup_csv_sampled_before_error_checking/full_data_beforecheck_forcomparisontoupdated.csv', 
                            header=TRUE, stringsAsFactors=FALSE, row.names=1)
hw <- read.csv(file='backup_csv_sampled_before_error_checking/sample_lines_to_check.csv', 
                            header=TRUE, stringsAsFactors=FALSE, row.names=1)
js <- read.csv(file='backup_csv_sampled_before_error_checking/sample_lines_reviewer_to_check.csv', 
                            header=TRUE, stringsAsFactors=FALSE, row.names=1)
current_data <- read.csv(file='Data/somalia_all_data_2020.csv', header=TRUE, stringsAsFactors=FALSE)
colnames(current_data)[1] <- 'Nat.case.ser.no'


In [35]:
head(current_data)
head(hw)

Nat.case.ser.no,Age,Sex,Date.onset.rash,Date.detected,Village,District,Region,National.outbreak.Number,Reg.outbreak.no,Adobe.page.ref,Comment,Corresponding.Outbreak.Info.page.number
<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<int>
1,70,F,1976-08-30,NA,,,Mogadishu (Banaadir),1,NA,226,,NA
10,6,F,1976-10-12,NA,,,Mogadishu (Banaadir),1,NA,226,,NA
1060,12,F,1977-04-01,NA,Bula Badid,Baidoa,Bay,NA,29,156,,NA
11,15,F,1976-10-06,NA,,,Mogadishu (Banaadir),1,NA,226,,NA
119,NA,,NA,NA,,,,NA,NA,219,NOT SMALLPOX,NA
1191,NA,,NA,NA,,,,NA,NA,148,,NA


,Nat.case.ser.no,Age,Sex,Date.onset.rash,Date.detected,Village,District,Region,National.outbreak.Number,Reg.outbreak.no,Adobe.page.ref,Comment,Corresponding.Outbreak.Info.page.number
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<int>
2869,652,14,M,1977-04-01,1977-05-25,Duboy,Kansadere,Bay,199,78,183,,NA
10,1004,70,M,1977-05-25,1977-05-31,Gubey,Wajid,Bakool,308,39,160,,NA
941,1845,22,M,1977-04-09,1977-06-19,Bilakey (2),Wajid,Bakool,547,20,100,,NA
334,1297,3,M,1977-04-15,1977-06-13,Mundul Jall,Bardere,Gedo,NA,7,140,,NA
2660,464,48,F,1977-05-08,1977-05-09,Warurie,Baidoa,Bay,73,31,196,,NA
1340,2205,NA,,NA,NA,Mora Garad,Huddum,Bakool,NA,NA,75,,NA


In [36]:
# hw and js should be exact copies of 20% of sample_precheck
dim(hw)
dim(js)
dim(precheck_data)
dim(current_data)

[1] 326  13

[1] 326  13

[1] 3253   13

[1] 3253   13

In [136]:
error_rate <- function(pre_sample_df, cur_full_df){
    # extract subset of full_db using Nat.case.ser.no from sample_db
    subset_df <- cur_full_df[cur_full_df$Nat.case.ser.no %in% pre_sample_df$Nat.case.ser.no,]
    # rearrange dataframe by Nat.case.ser.no
    subset_df <- subset_df[order(subset_df$Nat.case.ser.no),]
    pre_sample_df <- pre_sample_df[order(pre_sample_df$Nat.case.ser.no),]
    # if Nat.case.ser.no do not match, then there is an error somewhere 
    # case numbers may have been corrected in the cleaning
    if (!all(pre_sample_df$Nat.case.ser.no == subset_df$Nat.case.ser.no)) {
        print('Ordered Nat.case.ser.no lists do not match')
    }
    # convert NA to -1 to allow comparison
    subset_df[is.na(subset_df)] <- -1
    pre_sample_df[is.na(pre_sample_df)] <- -1
    # test for equality
    comparison <- subset_df == pre_sample_df
    # count errors 
    tf <- table(comparison)
    tf_1to8 <- table(comparison[,-(9:13)])
    # total error percentage
    print(paste('total error %:', tf[[1]]/(tf[[1]]+tf[[2]])*100))
    print(paste('error for cols 1 to 8 %:', tf_1to8[[1]]/(tf_1to8[[1]]+tf_1to8[[2]])*100))
    # number of lines containing errors
    print(paste(sum(rowSums(1-comparison)!=0), 'lines out of', length(rowSums(1-comparison)), 'contain errors'))
    print('number of errors per column')
    print(colSums(1-comparison))
    return (comparison)
}

In [137]:
hw_comp <- error_rate(hw, current_data)

[1] "total error %: 1.76970268994809"
[1] "error for cols 1 to 8 %: 1.57208588957055"
[1] "61 lines out of 326 contain errors"
[1] "number of errors per column"
                        Nat.case.ser.no                                     Age 
                                      0                                      10 
                                    Sex                         Date.onset.rash 
                                      1                                      17 
                          Date.detected                                 Village 
                                     11                                       2 
                               District                                  Region 
                                      0                                       0 
               National.outbreak.Number                         Reg.outbreak.no 
                                     20                                       9 
                         Adob

In [138]:
js_comp <- error_rate(js, current_data)

[1] "total error %: 1.27418593676262"
[1] "error for cols 1 to 8 %: 1.3420245398773"
[1] "42 lines out of 326 contain errors"
[1] "number of errors per column"
                        Nat.case.ser.no                                     Age 
                                      0                                       5 
                                    Sex                         Date.onset.rash 
                                      1                                       8 
                          Date.detected                                 Village 
                                     12                                       5 
                               District                                  Region 
                                      2                                       2 
               National.outbreak.Number                         Reg.outbreak.no 
                                      6                                       9 
                         Adobe